### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from linearmodels.iv import IV2SLS

import warnings
warnings.filterwarnings("ignore")

### Load the data

In [2]:
data = pd.read_excel('Project1Data.xlsx')

In [3]:
# Create log-transformed variables for demand and supply analysis
data['ln_qu'] = np.log(data['qu'])
data['ln_cprice'] = np.log(data['cprice'])
data['ln_tprice'] = np.log(data['tprice'])
data['ln_bprice'] = np.log(data['bprice'])
data['ln_wprice'] = np.log(data['wprice'])
data['ln_income'] = np.log(data['incom'])

# Check the transformed data
data.head()

,maand,year,month,qu,cprice,tprice,oprice,incom,q1,q2,q3,q4,bprice,wprice,ln_qu,ln_cprice,ln_tprice,ln_bprice,ln_wprice,ln_income
0,Jan 1990,1990,1,0.55,12.12,18.6,1.0,1640.87,1,0,0,0,3.47,28.15,-0.597837,2.494857,2.923162,1.244155,3.337547,7.402982
1,Feb 1990,1990,2,0.65,12.12,18.6,1.0,1538.60,1,0,0,0,3.40,28.15,-0.430783,2.494857,2.923162,1.223775,3.337547,7.338628
2,Mar 1990,1990,3,0.66,12.12,18.6,1.0,1680.93,1,0,0,0,3.26,28.33,-0.415515,2.494857,2.923162,1.181727,3.343921,7.427102
3,Apr 1990,1990,4,0.66,12.12,18.6,1.0,1656.20,0,1,0,0,3.46,28.49,-0.415515,2.494857,2.923162,1.241269,3.349553,7.412281
4,May 1990,1990,5,0.64,12.12,18.6,1.0,1700.80,0,1,0,0,3.47,28.55,-0.446287,2.494857,2.923162,1.244155,3.351657,7.438854


#### Demand equation

##### Step 1. Using the OLS method 

In [4]:
# Define the independent variables (including season dummies and other factors)
X = sm.add_constant(data[['ln_cprice', 'ln_income', 'ln_tprice', 'q1', 'q2', 'q3']])

# Dependent variable: log of quantity demanded
y = data['ln_qu']

# Fit the OLS model for demand
demand_model = sm.OLS(y, X).fit()

# Print the summary of the regression
print(demand_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  ln_qu   R-squared:                       0.290
Model:                            OLS   Adj. R-squared:                  0.235
Method:                 Least Squares   F-statistic:                     5.239
Date:                Thu, 19 Sep 2024   Prob (F-statistic):           0.000144
Time:                        21:18:59   Log-Likelihood:                 79.838
No. Observations:                  84   AIC:                            -145.7
Df Residuals:                      77   BIC:                            -128.7
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.2635      1.997     -2.135      0.0

##### Step 2. Using first stage SLS to define the instrument (to check if the instrument variable would be a good fit or not)

In [10]:
# First stage regression: Regress ln_cprice on the instrument ln_bprice and other exogenous variables
X_first_stage = sm.add_constant(data[['ln_bprice', 'incom', 'q1', 'q2', 'q3']])
y_first_stage = data['ln_cprice']

first_stage_model = sm.OLS(y_first_stage, X_first_stage).fit()

# Print the summary to check the significance of ln_bprice in predicting ln_cprice
print(first_stage_model.summary())

                            OLS Regression Results                            
Dep. Variable:              ln_cprice   R-squared:                       0.947
Model:                            OLS   Adj. R-squared:                  0.944
Method:                 Least Squares   F-statistic:                     281.0
Date:                Thu, 19 Sep 2024   Prob (F-statistic):           2.33e-48
Time:                        21:23:12   Log-Likelihood:                 153.48
No. Observations:                  84   AIC:                            -295.0
Df Residuals:                      78   BIC:                            -280.4
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4777      0.051     28.761      0.0

##### Step 3. Using the 2SLS method to form the final demand equation

In [6]:
# First stage: Instrument price with bprice and wprice
X_iv = data[['incom','q1', 'q2', 'q3']]
Z_iv = sm.add_constant(data[['ln_bprice']])  # Instruments

# Second stage: Using predicted prices to estimate demand
iv_model = IV2SLS(dependent=data['ln_qu'], exog=X_iv, endog=data['ln_cprice'], instruments=Z_iv).fit()

# Print the summary of the IV regression
print(iv_model.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  ln_qu   R-squared:                      0.9457
Estimator:                    IV-2SLS   Adj. R-squared:                 0.9422
No. Observations:                  84   F-statistic:                    1762.2
Date:                Thu, Sep 19 2024   P-value (F-stat)                0.0000
Time:                        21:19:18   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
incom          0.0002  7.733e-05     2.6019     0.0093   4.965e-05      0.0004
q1            -0.1128     0.0328    -3.4402     0.00

----------------------------------------------

#### Supply equation

##### Step 1. Using first stage SLS to define the instrument (to check if the instrument variable would be a good fit or not)

In [11]:
# First Stage: Instrumenting ln_cprice with ln_bprice and ln_wprice
X_iv_supply = sm.add_constant(data[['ln_bprice', 'ln_wprice']])  # Instruments (exogenous variables)
y_first_stage_supply = data['ln_cprice']

first_stage_supply = sm.OLS(y_first_stage_supply, X_iv_supply).fit()

# Print the summary of the first stage to check relevance
print(first_stage_supply.summary())

                            OLS Regression Results                            
Dep. Variable:              ln_cprice   R-squared:                       0.953
Model:                            OLS   Adj. R-squared:                  0.952
Method:                 Least Squares   F-statistic:                     828.4
Date:                Thu, 19 Sep 2024   Prob (F-statistic):           1.18e-54
Time:                        21:25:26   Log-Likelihood:                 158.56
No. Observations:                  84   AIC:                            -311.1
Df Residuals:                      81   BIC:                            -303.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.7902      0.266     -2.968      0.0

##### Step 2. Using the 2SLS method to form the final demand equation

In [7]:
# Define exogenous variables for the supply function
X_iv_supply = data[['wprice', 'q1', 'q2', 'q3']]  # Exogenous variables for the supply equation

# Instrument: ln_bprice (coffee bean price)
Z_iv_supply = sm.add_constant(data[['ln_bprice']])  # Instruments (coffee bean price)

# Dependent variable: ln_qu (log of quantity supplied)
y_supply = data['ln_qu']  

# Second Stage: Using 2SLS to estimate the supply equation
iv_model_supply = IV2SLS(dependent=y_supply, exog=X_iv_supply, endog=data['ln_cprice'], instruments=Z_iv_supply).fit()

# Print the summary of the IV regression
print(iv_model_supply.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  ln_qu   R-squared:                      0.9447
Estimator:                    IV-2SLS   Adj. R-squared:                 0.9412
No. Observations:                  84   F-statistic:                    1709.1
Date:                Thu, Sep 19 2024   P-value (F-stat)                0.0000
Time:                        21:19:41   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
wprice         0.0138     0.0062     2.2262     0.0260      0.0016      0.0259
q1            -0.1323     0.0303    -4.3628     0.00